In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.stats import pearsonr
import shutil

In [3]:
T=pd.read_excel('../raw_data/climate_change_download_0.xls',index_col=1,na_values='..')
T.columns=T.columns.astype(str)
T=T.loc[T['Series name']=='Average daily min/max temperature (1961-1990, Celsius)','2011']

T=T.dropna().map(lambda s: s.split('/ ')[1]).astype(float)
T.name='Temperature'
T.rename(index={'Iran, Islamic Rep.':'Iran', 
                'Russian Federation':'Russia',
               'Korea, Dem. Rep.':'South Korea',
               'Hong Kong SAR, China':'Hong Kong',
               'Slovak Republic':'Slovakia',
               'United States':'United States of America',
               'Venezuela, RB':'Venezuela'},inplace=True)

In [4]:
H= pd.read_excel('../raw_data/Hip_fracture.xlsx',index_col=0)
H.rename(index={'United States':'United States of America'},inplace=True)

In [5]:


Latitude= pd.read_excel('../raw_data/Latitudes.xlsx',index_col=0)
Latitude['latitude']=Latitude.Latitude.str[:2].astype(int)
Latitude.loc[(Latitude.Latitude.str[-1]=='S'),'latitude']*=-1

Latitude.rename(index={'Iran (Islamic Republic of)':'Iran',
                       'Russian Federation':'Russia'
                      },inplace=True)

D= H.join(T)
D['Latitude']= Latitude.latitude.abs()

In [6]:
D.loc[D.Latitude.isnull()].index

Index(['Hong Kong', 'Japan', 'Singapore', 'South Korea', 'Taiwan', 'Serbia',
       'Morocco'],
      dtype='object', name='Country')

In [7]:
missing_latitude= {'Singapore':1,'Japan':35,'Hong Kong':22,
                  'South Korea':38, 'Serbia':45, 'Morocco':34,'Taiwan':25
                 }

D.loc[missing_latitude.keys(),'Latitude']= pd.Series(missing_latitude)

In [8]:
D

,Continent,Men,Women,Total,Temperature,Latitude
Country,,,,,,
Nigeria,Africa,2,2,2.0,32.8,9.0
South Africa,Africa,17,20,19.0,24.9,25.0
China,Asia,102,182,141.0,13.0,39.0
Hong Kong,Asia,149,324,230.0,26.0,22.0
India,Asia,122,172,147.0,29.6,28.0
...,...,...,...,...,...,...
Tunisia,Middle East,41,58,50.0,24.9,36.0
Canada,North America,136,298,211.0,-0.6,45.0
United States of America,North America,155,339,240.0,14.9,39.0


In [9]:
D.loc[D.Temperature.isnull()].index

Index(['Taiwan', 'Serbia'], dtype='object', name='Country')

In [10]:
H.index.difference(T.index)

Index(['Serbia', 'Taiwan'], dtype='object')

In [11]:
vitD= pd.read_table('../data/VitaminD_per_country.tsv',index_col=0,squeeze=True)
vitD.rename(index={'USA':'United States of America','Korea (South)':'South Korea'},inplace=True)
D['VitaminD']=vitD

vitD.index.difference(D.index)

Index(['Cameroon', 'Estonia', 'Fidji Islands', 'Gambia', 'Vietnam'], dtype='object')

In [12]:
D.loc[D.VitaminD.isnull()].index

Index(['Nigeria', 'Hong Kong', 'Indonesia', 'Philippines', 'Singapore',
       'Croatia', 'Hungary', 'Lithuania', 'Malta', 'Portugal', 'Romania',
       'Serbia', 'Slovakia', 'Slovenia', 'Turkey', 'Argentina', 'Chile',
       'Colombia', 'Ecuador', 'Mexico', 'Venezuela', 'Kuwait', 'Morocco',
       'Oman'],
      dtype='object', name='Country')

# Calcium

In [13]:
C= pd.read_excel('../raw_data/Calcium.xlsx')


In [14]:
C.index= C.Country.map(lambda s: s.split('[')[0].strip())
assert ~C.index.duplicated().any()
C.rename(index={'USA':'United States of America',
               'UK':'United Kingdom'},inplace=True)

In [15]:
D['Calcium']= C.iloc[:,0]

In [16]:
D.loc[D.Calcium.isnull()].index

Index(['Hong Kong', 'Lithuania', 'Malta', 'Poland', 'Romania', 'Slovakia',
       'Slovenia', 'Turkey', 'Venezuela', 'Lebanon', 'Oman', 'Saudi Arabia',
       'Tunisia'],
      dtype='object', name='Country')

In [17]:
C.index.difference(D.index)

Index(['Algeria', 'Bangladesh', 'Barbados', 'Bolivia', 'Botswana',
       'Burkina Faso', 'Cameroon', 'Cape Verde', 'Costa Rica', 'Egypt',
       'Ethiopia', 'Gambia', 'Guam', 'Jamaica', 'Latvia', 'Mali', 'Myanmar',
       'Nepal', 'Pakistan', 'Palestine', 'Polandi', 'Tanzania', 'Tonga',
       'Uganda', 'Vietnam'],
      dtype='object', name='Country')

In [29]:
L= pd.read_excel('../raw_data/Light_city_29.1.20.xlsx',index_col=[0,1])
assert L.index.is_unique


C.rename(index={'United States':'United States of America'},inplace=True)

D['Sunlight']=L.Year.groupby(level=0).mean()

In [30]:
D.loc[D.Sunlight.isnull()].index

Index(['Malaysia', 'Jordan', 'Kuwait', 'Lebanon', 'United States of America'], dtype='object', name='Country')

In [31]:
D.to_excel('../data/Data_for_correlations.xlsx')


In [32]:
(~D.isnull()).sum()

Continent      62
Men            62
Women          62
Total          61
Temperature    60
Latitude       62
VitaminD       38
Calcium        49
Sunlight       57
dtype: int64